# TUTORIAL PYTORCH
## Autograd
###### Fuente: [Documentación Oficial de Pytorch](https://pytorch.org/tutorials/)
###### Edición y traducción por Cristobal Donoso O. 
###### Agosto 2018

In [1]:
import torch

### Autograd
La clase principal para todas las redes neuronales es **autograd**. Este paquete nos permite diferenciar automaticamente todas las operaciones en los Tensores. El backprop estará definido según la estructura de nuestro codigo; para cada iteración puede ser diferente.

#### Tensor
La clase principal de autograd es ```torch.Tensor```. Si inicializas el atributo ```.requires_grad``` como ```True``` entonces comenzará a hacer seguimiento de todas las operaciones en el. 

In [78]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Cuando finalices el *foward* puedes llamar a ```.backwar()``` y obtendras todos los gradientes automaticamente. El gradiente para un tensor será guardado en un atributo ```.grad```<br><br>
Para sacar un tensor del historial de seguimiento puedes llamar al metodo ```.detach()```<br>
En ocaciones, no es necesario ajustar pesos del modelo (por ejemplo en el testing). Para detener todo el historial de seguimiento basta con colocar ```torch.no_grad()```.

#### Función
Tensores y Funciones están interconectados. Juntos, construyen el grafo aciclico de operaciones y su historial de seguimiento. Cada Tensor tiene asociado un atributo ```.grad_fn``` que hace referencia a la clase **Function**  que ha creado al tensor. Si el tensor fue creado por el usuario ```.grad_fn``` será ```none```

In [79]:
print(x.grad_fn)

None


In [80]:
y = x + 2
print(y,'\n','-'*100)
print('Puntero referencia a Function Class de y:',y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>) 
 ----------------------------------------------------------------------------------------------------
Puntero referencia a Function Class de y: <AddBackward object at 0x7fd048eb02e8>


En este caso la variable ```y``` fué creada como resultado de una operación, por lo tanto tiene un ```.grad_fn```. Podemos seguir realizando operaciones y generaremos más tensores

In [81]:
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>)
tensor(27., grad_fn=<MeanBackward1>)


Si existe algun Tensor que **no fue inicializado** con ```requires_grad=True``` simplemente cambiamos el valor por defecto en la clase.

In [82]:
a = torch.randn(2, 2)   #creamos el tensor
a = ((a * 3) / (a - 1)) # aplicamos operaciones sobre el
print(a.requires_grad)  # vemos si se le está haciendo seguimiento a los gradientes
a.requires_grad_(True)  # Cambiamos el valor del atributo
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


#### Gradientes
Como la variable ```out``` contiene un *escalar*, ```out.backward()``` es equivalente con ```out.backward(torch.tensor(1))```

In [83]:
out.backward()

Matemáticamente,<br>
<center>$
\begin{equation}
out = \frac{1}{4}\sum_i z_i\\
z_i = 3(y_i)^2\\
y_i = x_i + 2\\
\end{equation}
$</center>
En este caso
<center>$
x = \begin{bmatrix}
    1 & 1  \\
    1 & 1
\end{bmatrix}
$</center>
<br>entonces la evaluación de cada elemento en x sobre el grafo de operaciones arrojará como resultado:
<center>$
z_i\Big|_{x_i=1} = 27
$</center>
Finalmente hacemos el calculo de los gradientes asociados,
<center>$
\frac{\partial out}{\partial x_i} = \frac{3}{2}(x_i+2) 
$</center>
donde 
<center>
    $\frac{\partial out}{\partial x_i} \Big|_{x_i=1}=\frac{9}{2} = 4.5$
</center>

In [84]:
print('Valor de la variable')
print(z)
print('-'*100,'\nValor de los gradientes')
print(x.grad)

Valor de la variable
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>)
---------------------------------------------------------------------------------------------------- 
Valor de los gradientes
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


#### Otro ejemplo

In [85]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y.data)

tensor([-1405.2299,  -287.5595,  -637.3141])


Note que acá estamos inicializando los gradientes, puesto que ```y``` no es escalar

In [86]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)
print(x.grad)

tensor([ 102.4000, 1024.0000,    0.1024])


Podemos detener todo el seguimiento de gradientes usando ```torch.no_grad()```

In [77]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
